In [ ]:
import logging; logging.basicConfig(level=logging.DEBUG)
import time
import datetime as dt

from dropbot import SerialProxy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
from dropbot.hardware_test import time_it

%matplotlib inline


@time_it
def stress_test_number_of_channels(voltage=120, max_channels=10, n_cycles=3):
    number_of_channels = proxy.number_of_channels
    
    proxy.hv_output_enabled = True
    proxy.hv_output_selected = True
    measured_voltage = []
    measured_capacitance = []
    measured_temperature = []
    rms_input_current = []
    max_input_current = []
    rms_output_current = []
    max_output_current = []

    # wait for the voltage to stabilize
    proxy.voltage = voltage
    time.sleep(0.5)

    for j in range(n_cycles):
        for i in range(max_channels):
            clear_output()
            display('%d/%d, repeat %d/%d' % (i + 1, max_channels, j + 1, n_cycles))
            state = np.zeros(number_of_channels, dtype=np.int)
            state[:i] = 1
            proxy.state_of_channels = state
            measured_voltage.append(proxy.measure_voltage())
            measured_capacitance.append(proxy.measure_capacitance())
            measured_temperature.append(proxy.measure_temperature())
            results = proxy.measure_input_current()
            rms_input_current.append(results['rms'])
            max_input_current.append(results['max'])
            results = proxy.measure_output_current()
            rms_output_current.append(results['rms'])
            max_output_current.append(results['max'])

    state = np.zeros(number_of_channels, dtype=np.int)

    measured_voltage = np.array(measured_voltage)
    return {'measured_voltage': measured_voltage,
            'rms_input_current': rms_input_current,
            'max_input_current': max_input_current,
            'rms_output_current': rms_output_current,
            'max_output_current': max_output_current,
            'measured_capacitance': measured_capacitance,
            'measured_temperature': measured_temperature,
           }


@time_it
def stress_test_cycle_channels(voltage=120, n_channels=10, n_cycles=3):
    max_channels = proxy.number_of_channels
    
    proxy.hv_output_enabled = True
    proxy.hv_output_selected = True
    measured_voltage = []
    measured_capacitance = []
    measured_temperature = []
    rms_input_current = []
    max_input_current = []
    rms_output_current = []
    max_output_current = []

    # wait for the voltage to stabilize
    proxy.voltage = voltage
    time.sleep(0.5)

    try:
        for j in range(n_cycles):
            for i in range(max_channels):
                clear_output()
                display('%d/%d, repeat %d/%d' % (i + 1, max_channels, j + 1, n_cycles))
                state = np.zeros(max_channels, dtype=np.int)
                if i + n_channels < max_channels:
                    state[i:i + n_channels] = 1
                else:
                    state[i:] = 1
                    state[:i + n_channels - max_channels] = 1

                proxy.state_of_channels = state
                measured_voltage.append(proxy.measure_voltage())
                measured_capacitance.append(proxy.measure_capacitance())
                measured_temperature.append(proxy.measure_temperature())
                results = proxy.measure_input_current()
                rms_input_current.append(results['rms'])
                max_input_current.append(results['max'])
                results = proxy.measure_output_current()
                rms_output_current.append(results['rms'])
                max_output_current.append(results['max'])
    except ValueError:
        print 'Error: Test did not complete.'

    measured_voltage = np.array(measured_voltage)
    return {'measured_voltage': measured_voltage,
            'rms_input_current': rms_input_current,
            'max_input_current': max_input_current,
            'rms_output_current': rms_output_current,
            'max_output_current': max_output_current,
            'measured_capacitance': measured_capacitance,
            'measured_temperature': measured_temperature,
            'n_channels': n_channels
           }


def plot_results(results, xlabel):
    print 'Test took %.1f s' % results['duration']

    plt.figure()
    plt.plot(results['rms_input_current'], label='rms')
    plt.plot(results['max_input_current'], label='max')
    plt.xlabel(xlabel)
    plt.ylabel('Input current (A)')
    plt.legend()

    plt.figure()
    plt.plot(results['rms_output_current'], label='rms')
    plt.plot(results['max_output_current'], label='max')
    plt.xlabel(xlabel)
    plt.ylabel('Output current (A)')
    plt.legend()

    plt.figure()
    plt.plot(results['measured_voltage'])
    plt.xlabel(xlabel)
    plt.ylabel('Output voltage (V)')

    plt.figure()
    plt.plot(results['measured_temperature'])
    plt.xlabel(xlabel)
    plt.ylabel('Microcontroller temperature ($\circ$C)')

    plt.figure()
    plt.plot(results['measured_capacitance'])
    plt.xlabel(xlabel)
    plt.ylabel('Capacitance (pF)')

In [ ]:
try:
    proxy.terminate()
except:
    pass

# Connect to the DropBot and print it's properties
proxy = SerialProxy()
proxy.properties

In [ ]:
results = stress_test_number_of_channels(voltage=140, max_channels=60)
plot_results(results, '# channels actuated')

In [ ]:
results = stress_test_cycle_channels(voltage=140, n_channels=60, n_cycles=1)
plot_results(results, '#')